### TODO
1. Combine to three classes
2. Excuate

In [3]:
file_path = "/nfs/research/goldman/zihao/Datas/p2_compViridian_P2/Folder_1_dataPrep/Folder_alignment/Col_MAPLE_format_consensuses.txt"
data_set = set()

with open(file_path, 'r') as file:
    for line in file:
        if line.startswith(">"):
            data_set.add(line.strip())

len(data_set)

136344

## 1.forCoverage

## 2.sampling

```bash
bsub -M 20000 -e /nfs/research/goldman/zihao/TEST/errorChecking.txt -o /nfs/research/goldman/zihao/TEST/outputChecking.txt 'python3 /nfs/research/goldman/zihao/TEST/allPos.py'
```

In [19]:
import pandas as pd
pd.read_csv('N-dataProcessing_allPos.txt',sep = '\t').head(20)

,ID,Position,N,Cov_RATIO
0,ERR6542228,1,A,0.000000
1,SRR23130330,1,A,0.000000
2,ERR7897214,1,A,0.000000
3,ERR6499101,1,A,0.000000
4,ERR4905438,1,A,0.000225
5,ERR6483047,2,T,0.000214
6,ERR7897214,2,T,0.000000
7,ERR6524376,2,T,0.000000
8,SRR20724513,2,T,0.000000
9,ERR6682766,2,T,0.000000


In [16]:
import os
import random
from multiprocessing import Pool
import time

def check_files_with_id(folder_path, checkid_file, max_copy=50):
    id_set = set()
    with open(checkid_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                id_set.add(line[1:])
    count = 0
    selected_files = []
    for filename in os.listdir(folder_path):
        if any(id_str in filename for id_str in id_set):
            selected_files.append(os.path.join(folder_path, filename))
            count += 1
        if count >= max_copy:
            break
    return selected_files

class ReservoirSampler:
    def __init__(self, k):
        self.k = k

    def generator(self, file_name):
        with open(file_name, 'r') as file:
            file_id = os.path.splitext(os.path.basename(file_name))[0]
            file_id = file_id.replace('_coverage', '')
            next(file)
            for line in file:
                yield [file_id] + line.strip().split('\t')

    def reservoir_sampling(self, stream):
        reservoir = {}
        for item in stream:
            position = int(item[1])
            if position not in reservoir:
                reservoir[position] = [item]
            elif len(reservoir[position]) < self.k:
                reservoir[position].append(item)
            else:
                j = random.randint(0, len(reservoir[position]) + 1)
                if j < self.k:
                    reservoir[position][j] = item
        return reservoir

def process_file(file_name):
    sampler = ReservoirSampler(5)
    stream = sampler.generator(file_name)
    samples = sampler.reservoir_sampling(stream)
    return samples

def merge_samples(all_samples_list, k):
    all_samples = {}
    for samples in all_samples_list:
        for position, sample_list in samples.items():
            if position not in all_samples:
                all_samples[position] = sample_list
            else:
                all_samples[position].extend(sample_list)
                if len(all_samples[position]) > k:
                    random.shuffle(all_samples[position])
                    all_samples[position] = all_samples[position][:k]
    return all_samples

def main():
    folder_path_1 = "/nfs/research/goldman/zihao/Datas/p1/File_5_coverage/Decompress/"
    checkid_file = "/nfs/research/goldman/zihao/Datas/p2_compViridian_P2/Folder_1_dataPrep/Folder_alignment/Col_MAPLE_format_consensuses.txt"
    output_file_path = "N-dataProcessing_allPos.txt"

    selected_files = check_files_with_id(folder_path_1, checkid_file)

    with Pool() as pool:
        all_samples_list = pool.map(process_file, selected_files)

    all_samples = merge_samples(all_samples_list, 5)

    with open(output_file_path, 'w') as output_file:
        output_file.write('ID\tPosition\tN\tCov_RATIO\n')
        for position, sample_list in all_samples.items():
            for sample in sample_list:
                output_file.write('\t'.join(sample) + '\n')

if __name__ == "__main__":
    start_time = time.time()
    main()
    end_time = time.time()
    print(end_time - start_time)

16.214893341064453


## 3.forAnnot【ing】

In [ ]:
import os
import random
from multiprocessing import Pool
import time

def check_files_with_id(folder_path, checkid_file, max_copy=50):
    id_set = set()
    with open(checkid_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                id_set.add(line[1:])
    count = 0
    selected_files = []
    for filename in os.listdir(folder_path):
        if any(id_str in filename for id_str in id_set):
            selected_files.append(os.path.join(folder_path, filename))
            count += 1
        if count >= max_copy:
            break
    return selected_files

class ReservoirSampler:
    def __init__(self, k):
        self.k = k

    def generator(self, file_name):
        with open(file_name, 'r') as file:
            file_id = os.path.splitext(os.path.basename(file_name))[0]
            file_id = file_id.replace('_coverage', '')
            next(file)
            for line in file:
                yield [file_id] + line.strip().split('\t')

    def reservoir_sampling(self, stream):
        reservoir = {}
        for item in stream:
            position = int(item[1])
            if position not in reservoir:
                reservoir[position] = [item]
            elif len(reservoir[position]) < self.k:
                reservoir[position].append(item)
            else:
                j = random.randint(0, len(reservoir[position]) + 1)
                if j < self.k:
                    reservoir[position][j] = item
        return reservoir

def process_file(file_name):
    sampler = ReservoirSampler(5)
    stream = sampler.generator(file_name)
    samples = sampler.reservoir_sampling(stream)
    return samples

def merge_samples(all_samples_list, k):
    all_samples = {}
    for samples in all_samples_list:
        for position, sample_list in samples.items():
            if position not in all_samples:
                all_samples[position] = sample_list
            else:
                all_samples[position].extend(sample_list)
                if len(all_samples[position]) > k:
                    random.shuffle(all_samples[position])
                    all_samples[position] = all_samples[position][:k]
    return all_samples

def main():
    folder_path_1 = "/nfs/research/goldman/zihao/Datas/p1/File_5_coverage/Decompress/"
    checkid_file = "/nfs/research/goldman/zihao/Datas/p2_compViridian_P2/Folder_1_dataPrep/Folder_alignment/Col_MAPLE_format_consensuses.txt"

    selected_files = check_files_with_id(folder_path_1, checkid_file)

    with Pool() as pool:
        all_samples_list = pool.map(process_file, selected_files)

    all_samples = merge_samples(all_samples_list, 5)

    # Pass the all_samples data to the DataProcessor
    data_folder = "/nfs/research/goldman/zihao/Datas/p1/File_5_annot/New_Decompress/"
    output_file = "ANNOT_dataProcessing_allPos.txt"
    #"/nfs/research/goldman/zihao/Datas/p1_errorsProject_NEW/Folder_dataProcessing/ANNOT_dataProcessing_allPos.txt"

    processor = DataProcessor(all_samples, data_folder, output_file)
    processor.write_output()

class DataProcessor:
    def __init__(self, data1, data_folder, output_file):
        self.data1 = data1
        self.data_folder = data_folder
        self.output_file = output_file
    
    def read_data1(self):
        data1 = []
        for position, sample_list in self.data1.items():
            for sample in sample_list:
                # Ensure all elements in sample are strings
                data1.append([str(element) for element in sample])
        header = 'ID\tPosition\tN\tCov_RATIO\tAF\tSB\n'
        return header, data1
    
    def process_data(self):
        header, data1 = self.read_data1()

        for row in data1:
            id = row[0]
            filename = os.path.join(self.data_folder, f"{id}_annot.txt")
            if os.path.exists(filename):
                with open(filename) as f:
                    for line in f:
                        line = line.strip().split()
                        # Convert row[1] to string for comparison
                        if line[0] == str(row[1]):
                            row.append(line[3])
                            row.append(line[4])
                            break
                    else:
                        row.append("")
                        row.append("")
            else:
                row.append("")
                row.append("")

        return header, data1

    def write_output(self):
        """Write the processed data1 to the output file."""
        header, data1 = self.process_data()

        with open(self.output_file, "w") as f:
            f.write(header)
            for row in data1:
                f.write("\t".join(row) + "\n")
                
if __name__ == "__main__":
    start_time = time.time()
    main()
    end_time = time.time()
    print(end_time - start_time)

In [19]:
import os

class DataProcessor:
    def __init__(self, data1_file, data_folder, output_file):
        self.data1_file = data1_file
        self.data_folder = data_folder
        self.output_file = output_file
    
    def read_data1(self):
        """Read data1 from file and return the header and data."""
        data1 = []
        with open(self.data1_file) as f:
            header = f.readline().strip() + "\tAF\tSB\n"
            for line in f:
                line = line.strip().split()
                data1.append(line)
        return header, data1
    

    def process_data(self):
        """Process data1 by looking up corresponding rows in data folder."""
        header, data1 = self.read_data1()

        for row in data1:
            id = row[0]
            filename = os.path.join(self.data_folder, f"{id}_annot.txt")
            if os.path.exists(filename):
                with open(filename) as f:
                    for line in f:
                        line = line.strip().split()
                        if line[0] == row[1]:
                            row.append(line[3])
                            row.append(line[4])
                            break
                    else:
                        row.append("")
                        row.append("")
            else:
                row.append("")
                row.append("")

        return header, data1

    def write_output(self):
        """Write the processed data1 to the output file."""
        header, data1 = self.process_data()

        with open(self.output_file, "w") as f:
            f.write(header)
            for row in data1:
                f.write("\t".join(row) + "\n")
if __name__ == '__main__':
    data1_file = sampler_output
    data_folder = "/nfs/research/goldman/zihao/Datas/p1/File_5_annot/New_Decompress/"
    output_file = "/nfs/research/goldman/zihao/Datas/p1_errorsProject_NEW/Folder_dataProcessing/ANNOT_dataProcessing_allPos.txt"

    processor = DataProcessor(data1_file, data_folder, output_file)
    processor.write_output()

```python
def read_data1(self):
    """Read data1 from file and return the header and data."""
    data1 = []
    with open(self.data1_file) as f:
        header = f.readline().strip() + "\tAF\tSB\n"
        count = 0  # 计数器
        for line in f:
            line = line.strip().split()
            data1.append(line)
            count += 1
            if count == 10:  # 达到10行后终止循环
                break
    return header, data1
```

#### Sampler [Unencapsulate]
```python
import os
import random

def generator(file_list):
    for file_name in file_list:
        with open(file_name, 'r') as file:
            next(file)  # skip the header line
            for line in file:
                yield line.strip().split('\t')  # split by tab

def reservoir_sampling(stream, k):
    # Initialize the reservoir for each position
    reservoir = {}
    for item in stream:
        position = int(item[0])  # Position is in the first column
        if position not in reservoir:
            reservoir[position] = [item]
        elif len(reservoir[position]) < k:
            reservoir[position].append(item)
        else:
            # Replace elements with gradually decreasing probability
            j = random.randint(0, len(reservoir[position]) + 1)
            if j < k:
                reservoir[position][j] = item
    return reservoir

# Get a list of all the files in the directory
folder_path = output_folder
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".txt")]

# Create the generator
stream = generator(file_list)

# Reservoir sampling
k = 5  # The number of items to sample for each position
samples = reservoir_sampling(stream, k)

# Output the samples to a file
with open('/nfs/research/goldman/zihao/code/output.txt', 'w') as output_file:
    for position, sample_list in samples.items():
        for sample in sample_list:
            output_file.write('\t'.join(sample) + '\n')
```

### [!!!] Combination

```bash
bsub -M 20000 -e /nfs/research/goldman/zihao/N_errorsProject_1/Folder_allPos_dataProcessing/Folder_Checking/errorChecking_allPos_dataProcessing.txt -o /nfs/research/goldman/zihao/N_errorsProject_1/Folder_allPos_dataProcessing/Folder_Checking/outputChecking_allPos_dataProcessing.txt 'python3 /nfs/research/goldman/zihao/N_errorsProject_1/Folder_allPos_dataProcessing/CODE_allPos_dataProcessing.py'
```

In [ ]:
import os
import shutil
import random

class CovFileProcessor:
    def __init__(self, folder_path, checkid_file, output_folder):
        self.folder_path = folder_path
        self.checkid_file = checkid_file
        self.output_folder = output_folder

    def run(self):
        id_set = set()

        with open(self.checkid_file, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>'):
                    id_set.add(line[1:])

        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)

        for filename in os.listdir(self.folder_path):
            if any(id_str in filename for id_str in id_set):
                shutil.copy(os.path.join(self.folder_path, filename), os.path.join(self.output_folder, filename))


class ReservoirSampler:
    def __init__(self, output_file_path, k):
        self.output_file_path = output_file_path
        self.k = k

    def generator(self, file_list):
        for file_name in file_list:
            with open(file_name, 'r') as file:
                file_id = os.path.splitext(os.path.basename(file_name))[0]
                file_id = file_id.replace('_coverage', '')
                next(file)
                for line in file:
                    yield [file_id] + line.strip().split('\t')

    def reservoir_sampling(self, stream):
        reservoir = {}
        for item in stream:
            position = int(item[1])
            if position not in reservoir:
                reservoir[position] = [item]
            elif len(reservoir[position]) < self.k:
                reservoir[position].append(item)
            else:
                j = random.randint(0, len(reservoir[position]) + 1)
                if j < self.k:
                    reservoir[position][j] = item
        return reservoir

    def output_samples(self, samples):
        with open(self.output_file_path, 'w') as output_file:
            output_file.write('ID\tPosition\tN\tCov_RATIO\n')
            for position, sample_list in samples.items():
                for sample in sample_list:
                    output_file.write('\t'.join(sample) + '\n')

    def process_files(self, folder_path):
        file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".txt")]
        stream = self.generator(file_list)
        samples = self.reservoir_sampling(stream)
        self.output_samples(samples)


class AnnotDataProcessor:
    def __init__(self, data1_file, data_folder, output_file):
        self.data1_file = data1_file
        self.data_folder = data_folder
        self.output_file = output_file

    def read_data1(self):
        data1 = []
        with open(self.data1_file) as f:
            header = f.readline().strip() + "\tAF\tSB\n"
            for line in f:
                line = line.strip().split()
                data1.append(line)
        return header, data1

    def process_data(self):
        header, data1 = self.read_data1()

        for row in data1:
            id = row[0]
            filename = os.path.join(self.data_folder, f"{id}_annot.txt")
            if os.path.exists(filename):
                with open(filename) as f:
                    for line in f:
                        line = line.strip().split()
                        if line[0] == row[1]:
                            row.append(line[3])
                            row.append(line[4])
                            break
                    else:
                        row.append("")
                        row.append("")
            else:
                row.append("")
                row.append("")
        return header, data1

    def write_output(self):
        header, data1 = self.process_data()

        with open(self.output_file, "w") as f:
            f.write(header)
            for row in data1:
                f.write("\t".join(row) + "\n")


if __name__ == '__main__':
    folder_path = "/nfs/research/goldman/zihao/Datas/p1/File_5_coverage/Decompress/"
    checkid_file = "/nfs/research/goldman/zihao/Datas/p2_comp_viridian/2_alignment/Col_MAPLE_format_consensuses.txt"
    output_folder = "/nfs/research/goldman/zihao/Datas/p1_errorsProject_NEW/Folder_dataProcessing/Folder_MAPLE_allPos"
    sampler_output = "/nfs/research/goldman/zihao/Datas/p1_errorsProject_NEW/Folder_dataProcessing/COV_dataProcessing_allPos.txt"
    data_folder = "/nfs/research/goldman/zihao/Datas/p1/File_5_annot/New_Decompress/"
    output_file = "/nfs/research/goldman/zihao/Datas/p1_errorsProject_NEW/Folder_dataProcessing/ANNOT_dataProcessing_allPos.txt"
    nb_sample = 10
    
    # Run the CovFileProcessor
    cov_processor = CovFileProcessor(folder_path, checkid_file, output_folder)
    cov_processor.run()


    def count_files(folder_path):
        if not os.path.isdir(folder_path):
            raise ValueError("Invalid folder path!")

        file_count = 0
        for _, _, files in os.walk(folder_path):
            file_count += len(files)

        return file_count

    folder_path = output_folder

    try:
        num_files = count_files(folder_path)
        print('=========================== information =========================== ')
        print(f"The number of samples is: {num_files}")
    except ValueError as e:
        print(str(e))

    
    # Run the ReservoirSampler
    sampler = ReservoirSampler(sampler_output, nb_sample)
    sampler.process_files(output_folder)
    
    # Run the AnnotDataProcessor
    processor = AnnotDataProcessor(sampler_output, data_folder, output_file)
    processor.write_output()
    
    with open(output_file, 'r') as file:
        line_count = sum(1 for line in file)
    print('=========================== information =========================== ')
    print(f"The total number of samples completed for sampling is: {line_count-1}")

### ERR&ALL
```bash
sh /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errANDall_dataProcessing/BASH_dataProcessing_errAndall.sh
```

In [ ]:
#!/bin/bash

# Submit job 1
echo "Submitting job for OUTPUT"
bsub -M 2000 -e /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errANDall_dataProcessing/Folder_Checking/errorChecking_errPos_dataProcessing_forOutput.txt -o /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errANDall_dataProcessing/Folder_Checking/outputChecking_errPos_dataProcessing_forOutput.txt 'python3 /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errPos_dataProcessing/CODE_errPos_dataProcessing_forOutput.py'
echo "Job for output has been submitted"

# Submit job 2
echo "Submitting job for ANNOT"
bsub -M 2000 -e /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errANDall_dataProcessing/Folder_Checking/errorChecking_errPos_dataProcessing_forAnnot.txt -o /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errANDall_dataProcessing/Folder_Checking/outputChecking_errPos_dataProcessing_forAnnot.txt 'python3 /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errPos_dataProcessing/CODE_errPos_dataProcessing_forAnnot.py'
echo "Job for ANNOT has been submitted"

# Submit job 3
echo "Submitting job for COV"
bsub -M 2000 -e /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errANDall_dataProcessing/Folder_Checking/errorChecking_errPos_dataProcessing_forCov.txt -o /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errANDall_dataProcessing/Folder_Checking/outputChecking_errPos_dataProcessing_forCov.txt 'python3 /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errPos_dataProcessing/CODE_errPos_dataProcessing_forCov.py'
echo "Job for COV has been submitted"

# Submit job 4
echo "Submitting job for all Positions"
bsub -M 20000 -e /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errANDall_dataProcessing/Folder_Checking/errorChecking_allPos_dataProcessing.txt -o /nfs/research/goldman/zihao/N_errorsProject_1/Folder_errANDall_dataProcessing/Folder_Checking/outputChecking_allPos_dataProcessing.txt 'python3 /nfs/research/goldman/zihao/N_errorsProject_1/Folder_allPos_dataProcessing/CODE_allPos_dataProcessing.py'
echo "Job for all Positions has been submitted"